In [1]:
import cv2
import math
import serial
import time
import mediapipe as mp
import numpy as np

In [2]:
def compute_exercise_angles(a, b, c):
    x1, y1, z1 = a
    x2, y2, z2 = b
    x3, y3, z3 = c
    # Calculate vectors CA and CB
    CA = np.array([x1-x3, y1-y3, z1-z3])
    CB = np.array([x2-x3, y2-y3, z2-z3])
    
    # Calculate dot product of CA and CB
    dot_product = np.dot(CA, CB)
    
    # Calculate magnitudes of vectors CA and CB
    magnitude_CA = np.linalg.norm(CA)
    magnitude_CB = np.linalg.norm(CB)
    
    # Calculate angle theta using the dot product
    theta = np.arccos(dot_product / (magnitude_CA * magnitude_CB))
    
    # Convert angle from radians to degrees
    theta_degrees = np.degrees(theta)
    
    return theta_degrees

In [3]:
def send_message(port, command):
    pass

In [4]:
font = cv2.FONT_HERSHEY_SIMPLEX

blue = (255, 127, 0)
red = (50, 50, 255)
green = (127, 255, 0)
dark_blue = (127, 20, 0)
light_green = (127, 233, 100)
yellow = (0, 255, 255)
pink = (255, 0, 255)
black = (0, 0, 0)

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# SQUAT
# knee, hip, shoulder -> condition 1 -> hip angle
# knee, hip -> condition 2 -> knee-hip height
# foot, knee -> condition 3 -> foot-knee width 

# PUSH UP
# wrist, elbow, shoulder -> condition 1 -> elbow angle
# ankle, hip, shoulder -> condition 2 -> body angle

In [6]:
def extract_squat_landmarks(landmarks, direction):
    if direction == "left":
        knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
        foot = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].z]
    if direction == "right":
        knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
        foot = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].z]
    return knee, hip, shoulder, foot

In [7]:
def extract_push_up_landmarks(landmarks, direction):
    if direction == "left":
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
        ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
        hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
    if direction == "right":
        wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z]
        elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z]
        shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
        ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]
        hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
    return wrist, elbow, shoulder, ankle, hip

In [8]:
def process_frame(frame, status):
    if status == "pre":
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
    if status == "post":
        frame.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    return image

In [9]:
def compute_squat(l_shoulder, r_shoulder, l_hip, r_hip, l_knee, r_knee, l_foot, r_foot):
    # condition 1
    l_hip_angle = compute_exercise_angles(l_knee, l_hip, l_shoulder)
    r_hip_angle = compute_exercise_angles(r_knee, r_hip, r_shoulder)
    hip_angle = (l_hip_angle + r_hip_angle) / 2

    # condition 2
    def compute_knee_hip_height(knee, hip):
        _, knee_y, _ = knee
        _, hip_y, _ = hip
        return round(abs(knee_y - hip_y), 3)
    l_knee_hip_height = compute_knee_hip_height(l_knee, l_hip)
    r_knee_hip_height = compute_knee_hip_height(r_knee, r_hip)

    # condition 3
    def compute_foot_knee_width(foot, knee):
        foot_x, _, _ = foot
        knee_x, _, _ = knee
        return round(abs(foot_x - knee_x), 3)
    l_foot_knee_width = compute_foot_knee_width(l_foot, l_knee)
    r_foot_knee_width = compute_foot_knee_width(r_foot, r_knee)

    return hip_angle, l_knee_hip_height, r_knee_hip_height, l_foot_knee_width, r_foot_knee_width

In [10]:
def isCorrectSquat(hip_angle, l_knee_hip_height, r_knee_hip_height, l_foot_knee_width, r_foot_knee_width):
    squat_guide = ""
    is_squat_performed = False
    are_conditions_met = (60 <= hip_angle <= 120 and l_knee_hip_height <= 0.2 and r_knee_hip_height <= 0.2 and l_foot_knee_width <= 0.1 and r_foot_knee_width <= 0.1)

    if are_conditions_met:
        is_squat_performed = True
    else:
        # build guide given the wrong conditions
        if hip_angle < 60:
            squat_guide += "Increase the height of the hip\n"
        if hip_angle > 120:
            squat_guide += "Decrease the height of the hip\n"
        if l_knee_hip_height > 0.2 or r_knee_hip_height > 0.2:
            squat_guide += "Keep the thigh horizontal to the floor\n"
        if l_foot_knee_width > 0.1 or r_foot_knee_width > 0.1:
            squat_guide += "Do not exceed the tip of the toe with your knee\n"
    
    return is_squat_performed, squat_guide

In [11]:
def compute_push_up(l_shoulder, r_shoulder, l_elbow, r_elbow, l_wrist, r_wrist, l_hip, r_hip, l_ankle, r_ankle):
    # Condition 1
    l_elbow_angle = compute_exercise_angles(l_wrist, l_elbow, l_shoulder)
    r_elbow_angle = compute_exercise_angles(r_wrist, r_elbow, r_shoulder)
    elbow_angle = (l_elbow_angle + r_elbow_angle) / 2

    # Condition 2
    l_body_angle = compute_exercise_angles(l_ankle, l_hip, l_shoulder)
    r_body_angle = compute_exercise_angles(r_ankle, r_hip, r_shoulder)
    body_angle = (l_body_angle + r_body_angle) / 2
    return elbow_angle, body_angle

In [12]:
def isCorrectPushUp(elbow_angle, body_angle):
    push_up_guide = ""
    is_push_up_performed = False

    if 70 <= elbow_angle <= 100 and 160 <= body_angle <= 200:
        is_push_up_performed = True
    else:
        if elbow_angle < 70:
            push_up_guide += "Increase the height of the elbow\n"
        if elbow_angle > 100:
            push_up_guide += "Decrease the height of the elbow\n"
        if body_angle < 160:
            push_up_guide += "Increase the height of the body\n"
        if body_angle > 200:
            push_up_guide += "Decrease the height of the body\n"
    return is_push_up_performed, push_up_guide

In [21]:
cap = cv2.VideoCapture(0)
posture = "squat"
repetitions = 0
is_done = False
# get width and height from the camera
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = process_frame(frame, "pre")
        results = pose.process(frame)
        frame = process_frame(frame, "post")
        try:
            landmarks = results.pose_landmarks.landmark
            if posture == "squat":
                l_knee, l_hip, l_shoulder, l_foot = extract_squat_landmarks(landmarks, "left")
                r_knee, r_hip, r_shoulder, r_foot = extract_squat_landmarks(landmarks, "right")
                hip_angle, l_knee_hip_height, r_knee_hip_height, l_foot_knee_width, r_foot_knee_width = compute_squat(l_shoulder, r_shoulder, l_hip, r_hip, l_knee, r_knee, l_foot, r_foot)
                # put on the frame the hip angle at the height of the left hip
                cv2.putText(frame, "Hip Angle: " + str(round(hip_angle, 2)),
                        (int(l_hip[0] * w), int(l_hip[1] * h)), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                # put on the frame the knee-hip height at the height of the left knee
                cv2.putText(frame, "Knee-Hip Height: " + str(round(l_knee_hip_height, 2)),
                        (int(l_knee[0] * w), int(l_knee[1] * h)), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                # put on the frame the foot-knee width at the height of the left foot
                cv2.putText(frame, "Foot-Knee Width: " + str(round(l_foot_knee_width, 2)),
                        (int(l_foot[0] * w), int(l_foot[1] * h)), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                is_exercise_performed, exercise_guide = isCorrectSquat(hip_angle, l_knee_hip_height, r_knee_hip_height, l_foot_knee_width, r_foot_knee_width)
            
            if posture == "pushup":
                l_wrist, l_elbow, l_shoulder, l_ankle, l_hip = extract_push_up_landmarks(landmarks, "left")
                r_wrist, r_elbow, r_shoulder, r_ankle, r_hip = extract_push_up_landmarks(landmarks, "right")
                elbow_angle, body_angle = compute_push_up(l_shoulder, r_shoulder, l_elbow, r_elbow, l_wrist, r_wrist, l_hip, r_hip, l_ankle, r_ankle)
                # put on the frame the elbow angle at the height of the left elbow
                cv2.putText(frame, "Elbow Angle: " + str(round(elbow_angle, 2)),
                        (int(l_elbow[0] * w), int(l_elbow[1] * h)), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                # put on the frame the body angle at the height of the left ankle
                cv2.putText(frame, "Body Angle: " + str(round(body_angle, 2)),
                        (int(l_ankle[0] * w), int(l_ankle[1] * h)), font, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                is_exercise_performed, exercise_guide = isCorrectPushUp(elbow_angle, body_angle)

            if is_exercise_performed and not is_done:
                is_done = True
            if is_exercise_performed and is_done:
                is_done = False
                repetitions += 1

            cv2.rectangle(frame, (0,0), (400,73), (245,117,16), -1)
            # Rep data
            cv2.putText(frame, 'REPETITIONS: '+str(repetitions), (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(frame, "GUIDE: ", 
                        (15,36), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
            if exercise_guide != "":
                for i, guide in enumerate(exercise_guide.split('\n')):
                    t = i+1
                    cv2.putText(frame, guide, 
                        (15,36 + t*12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
            
        except:
            pass
        # Render detections
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        cv2.imshow('Mediapipe Feed', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: impossibile aprire il file oggetto condiviso: File o directory non esistente (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: fai